# Generating a list of Animeplanet usernames
* Finds users that have been recently active on the site

In [ ]:
import re

In [ ]:
name = "get_recent_username"
source = "animeplanet"

In [ ]:
PROXY_PARTITION = "1,2"
%run WebEndpointBase.ipynb

In [ ]:
# if we rerunning the notebook, then resume execution where we last left off
usernames = set()
if os.path.exists("recent_usernames.txt"):
    with open("recent_usernames.txt") as f:
        usernames = {x.strip() for x in f.readlines() if x.strip()}


logger.info(f"Starting with {len(usernames)} stored usernames")

In [ ]:
url_index = 0

In [ ]:
def get_users():
    urls = [
        ("https://www.anime-planet.com/community/", "/users/"),
        ("https://www.anime-planet.com/forum/", "members/"),
    ]
    global url_index
    url_index = (url_index + 1) % len(urls)
    url, prefix = urls[url_index]
    response = call_api(url)
    if response.status_code in [404]:
        logger.warning(f"Error {response} received when handling {url}")
        return set()
    if not response.ok:
        logger.warning(f"Error {response} received when handling {url}")
        return set()
    urls = re.findall(prefix + """[^"/#%?.']+""", response.text)
    users = {x[len(prefix) :] for x in urls}
    return users

In [ ]:
while True:
    users = get_users()
    usernames |= users
    if should_save("users"):
        atomic_to_csv(sorted(list(usernames)), "recent_usernames.txt")
        logger.info(f"Successfully written {len(usernames)} users ")